In [2]:
import requests
import pandas as pd



In [3]:
def get_cycles_data(curp):
    BASE_CURP_URL = f"https://api.siged.sep.gob.mx/CoreServices/servicios/alumno/schoolCyclesByStudent/curp={curp}"
    response = requests.get(BASE_CURP_URL,verify=False)
    if response.status_code == 200:

        return response.json()
    print(response.status_code)
    return None


def get_grades_data(StudentCycleID):
    BASE_GRADES_URL = f"https://api.siged.sep.gob.mx/CoreServices/servicios/alumno/evaluationsByStudentCycle/idStudentCycle={StudentCycleID}"
    response = requests.get(BASE_GRADES_URL,verify=False)
    if response.status_code == 200:
        return response.json()
    print(response.status_code)
    return None

In [4]:
import requests
import pandas as pd
import time
from random import randint
import json



In [5]:
#Requesters
# Function to handle retries for API requests
def make_request_with_retry(url, max_retries=5, delay=1, backoff_factor=2):
    retries = 0
    while retries < max_retries:
        response = requests.get(url, verify=False)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 503:  # Service Unavailable
            retries += 1
            wait_time = delay * (backoff_factor ** retries) + randint(0, 2)  # Add randomness for backoff
            print(f"503 received. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        else:
            print(f"Error {response.status_code} for URL {url}. Retrying...")
            retries += 1
            time.sleep(delay)
    
    print(f"Failed to retrieve data after {max_retries} retries.")
    return None

# Fetch cycles data with retry logic
def get_cycles_data(curp):
    BASE_CURP_URL = f"https://api.siged.sep.gob.mx/CoreServices/servicios/alumno/schoolCyclesByStudent/curp={curp}"
    return make_request_with_retry(BASE_CURP_URL)

# Fetch grades data with retry logic
def get_grades_data(StudentCycleID):
    BASE_GRADES_URL = f"https://api.siged.sep.gob.mx/CoreServices/servicios/alumno/evaluationsByStudentCycle/idStudentCycle={StudentCycleID}"
    return make_request_with_retry(BASE_GRADES_URL)

In [6]:
#Processor

# Fetch student data and create DataFrame
def fetch_student_data(curp):
    # Get school cycle data
    cycles_data = get_cycles_data(curp)

    if cycles_data is None or 'students' not in cycles_data:
        print(f"No student data found for CURP: {curp}")
        return None
    
    student = cycles_data['students'][0]
    student_id = student['id']
    CURP = student['curp']
    student_name = student['name']
    
    # Prepare a list to collect the data
    data = []
    
    # Extract each cycle information
    for cycle in student['schoolCyclesDTO']:
        print(json.dumps(cycle, indent=4, ensure_ascii=False))
        StudentCycleID = cycle['idStudentCycle']
        level = cycle['level']
        grade = cycle['grade']
        school_cycle = cycle['schoolCycle']
        shift = cycle['shift']
        avg = cycle['avgAcreditation']
        
        # Get grades for each cycle
        grades_data = get_grades_data(StudentCycleID)
  
        if grades_data is None or 'evaluationStudentCycle' not in grades_data:
            print(f"No grade data found for StudentCycleID: {StudentCycleID}")
            continue
        
        # Loop through the periods and subjects
        cycles =  grades_data['evaluationStudentCycle']
        if len(cycles) == 0:
            print(f"No grade data found for StudentCycleID: {StudentCycleID}")
            continue
        
        periodos = cycles[0]['evaluationPeriodsDTO']
        for periodo in periodos:
            period = periodo['period']
            for subject_info in periodo['subjectsAssesedDTO']:
                subject_name = subject_info['subject']
                grade = subject_info['subjectGrade']
                
                # Collect relevant data
                data.append({
                    'CURP': curp,
                    'StudentName': student_name,
                    'Level': level,
                    'SchoolCycle': school_cycle,
                    'Subject': subject_name,
                    'GradeObtained': grade,
                    'Period': period,
                    'Shift': shift,
                })
        
        # Add a row for the average grade
        data.append({
            'CURP': curp,
            'StudentName': student_name,
            'Level': level,
            'SchoolCycle': school_cycle,
            'Subject': 'Promedio',
            'GradeObtained': avg,
            'Period': 'N/A',
            'Shift': shift,
        })

    # Create a DataFrame
    df = pd.DataFrame(data)
    return df

In [7]:
# Example usage:
curp = "AAAA000102MVZLLTA6"  # Replace with the CURP you want to test
student_data_df = fetch_student_data(curp)
print(student_data_df)

No student data found for CURP: AAAA000102MVZLLTA6
None


/Users/ulisesquevedo/miniconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.siged.sep.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [48]:
student_data_df.to_csv("student_data2.csv", index=False)